In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("./international_results/results.csv")

In [3]:
data.shape

(42650, 9)

In [4]:
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [11]:
country=pd.concat([data['home_team'],data['away_team']]).unique()

In [14]:
print("country: ",len(country) )

country:  313


In [15]:
A=country[0]
B=country[1]
print(A,B)

Scotland England


In [29]:
dfAB=data[(data['home_team']==A)& (data['away_team']==B)] 
dfBA=data[(data['home_team']==B)& (data['away_team']==A)] 

In [26]:
dfAB.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
13,1880-03-13,Scotland,England,5,4,Friendly,Glasgow,Scotland,False


In [48]:
dfA1=dfAB['home_score']
dfB1=dfAB['away_score']

In [30]:
dfBA.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
11,1879-04-05,England,Scotland,5,4,Friendly,London,England,False
17,1881-03-12,England,Scotland,1,6,Friendly,London,England,False


In [57]:
dfA2=dfBA['away_score']
dfB2=dfBA['home_score']

In [58]:
dfA=pd.concat([dfA1,dfA2],axis=0)
dfB=pd.concat([dfB1,dfB2],axis=0)

In [59]:
len(dfA),len(dfB)

(117, 117)

In [74]:
n_games=len(dfA)
win_A=0
win_B=0
draw_AB=0
for i in range(n_games):
    a=dfA.iloc[i]
    b=dfB.iloc[i]
    if (a>b):
        win_A+=1
    elif (a<b):
        win_B+=1
    else:
        draw_AB+=1

In [75]:
print(A, "win: ", win_A,"; " ,B,"win: ", win_B,"; draw: ", draw_AB)

Scotland win:  42 ;  England win:  48 ; draw:  27


In [76]:
win_A+=draw_AB/2
win_B+=draw_AB/2
print(A, "win: ", win_A,"; " ,B,"win: ", win_B)

Scotland win:  55.5 ;  England win:  61.5


In [77]:
print(A, " to ", B, win_B/n_games,"; " ,B," to ", A, win_A/n_games)

Scotland  to  England 0.5256410256410257 ;  England  to  Scotland 0.47435897435897434


In [154]:
def calculate_weight(A,B):
    dfAB=data[(data['home_team']==A)& (data['away_team']==B)]
    dfA1=dfAB['home_score']
    dfB1=dfAB['away_score']
    dfBA=data[(data['home_team']==B)& (data['away_team']==A)]
    dfA2=dfBA['away_score']
    dfB2=dfBA['home_score']
    
    dfA=pd.concat([dfA1,dfA2],axis=0)
    dfB=pd.concat([dfB1,dfB2],axis=0)
    
    n_games=len(dfA)
    if (n_games>0):
        win_A=0
        win_B=0
        draw_AB=0
        for i in range(n_games):
            a=dfA.iloc[i]
            b=dfB.iloc[i]
            if (a>b):
                win_A+=1
            elif (a<b):
                win_B+=1
            else:
                draw_AB+=1

        #win_A+=draw_AB/2
        #win_B+=draw_AB/2

        AtoB=win_B/n_games
        BtoA=win_A/n_games
    else:
        AtoB=0.0
        BtoA=0.0
    return (AtoB,BtoA)
    

In [155]:
A=country[0]
B=country[1]
print(A,B)
AtoB,BtoA=calculate_weight(A,B)
print(A, " to ", B, AtoB,"; " ,B," to ", A, BtoA)

Scotland England
Scotland  to  England 0.41025641025641024 ;  England  to  Scotland 0.358974358974359


In [156]:
n=len(country)
print("country: ",n)

country:  313


In [157]:
M=np.zeros((n,n))

In [158]:
for i in range(n):
    for j in range(i+1,n):
        A=country[i]
        B=country[j]
        AtoB,BtoA=calculate_weight(A,B)
        M[i,j]=AtoB
        M[j,i]=BtoA

In [159]:
np.save('weights_1.npy', M)

In [160]:
dfm=pd.DataFrame(M,columns=country,index=country)

In [161]:
dfm.head()

,Scotland,England,Wales,Northern Ireland,United States,Uruguay,Austria,Hungary,Argentina,Belgium,...,Timor-Leste,Myanmar,Parishes of Jersey,Chameria,Saint Helena,Asturias,Western Australia,Two Sicilies,Surrey,Malabo
Scotland,0.000000,0.410256,0.216981,0.157895,0.200000,0.500000,0.363636,0.444444,0.5,0.650000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
England,0.358974,0.000000,0.144231,0.070707,0.181818,0.454545,0.210526,0.217391,0.2,0.153846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wales,0.575472,0.653846,0.000000,0.276596,0.500000,0.500000,0.500000,0.363636,0.5,0.428571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Northern Ireland,0.663158,0.767677,0.478723,0.000000,0.500000,0.500000,0.461538,0.666667,1.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
United States,0.500000,0.727273,0.000000,0.500000,0.000000,0.166667,0.666667,0.333333,0.6,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
dfm.to_csv("Weights_1.csv")

In [163]:
M[:2,:2]

array([[0.        , 0.41025641],
       [0.35897436, 0.        ]])

In [164]:
#damping_factor d
damping_factor=0.15
N=len(M)
print(damping_factor,N)

0.15 313


In [165]:
M_sum=M.sum(axis=0)

In [166]:
M_sum[0]

35.73336002581578

In [167]:
W=M.T

In [168]:
P=np.zeros_like(W)
for j in range(N):
    s=W[:,j].sum()
    if (s!=0):
        P[:,j]=W[:,j]/s

In [169]:
Q=(1-damping_factor)*P+damping_factor/N

In [170]:
eigenvector=np.ones(N)/N

In [171]:
max_iteration=100
for i in range(max_iteration):
    x=np.dot(Q,eigenvector)
    eigenvector=x/np.linalg.norm(x)

In [172]:
df_eigenvector=pd.DataFrame(eigenvector,index=country,columns=['rank'])

In [173]:
df_rank = df_eigenvector.sort_values(by='rank', ascending=False)

In [174]:
df_rank.head(10)

,rank
Brazil,0.226585
Spain,0.194723
Germany,0.179337
Argentina,0.175728
France,0.174624
England,0.170320
Netherlands,0.169864
Italy,0.159700
Hungary,0.152383
Mexico,0.147967


In [175]:
df_rank.to_csv('Rank.csv')